In [118]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>

<article class="day-desc read-aloud"><h2>--- Day 23: Amphipod ---</h2><p>A group of <a href="https://en.wikipedia.org/wiki/Amphipoda" target="_blank">amphipods</a> notice your fancy submarine and flag you down. "With such an impressive shell," one amphipod <span title="What? You didn't know amphipods can talk?">says</span>, "surely you can help us with a question that has stumped our best scientists."</p>
<p>They go on to explain that a group of timid, stubborn amphipods live in a nearby burrow. Four types of amphipods live there: <em>Amber</em> (<code>A</code>), <em>Bronze</em> (<code>B</code>), <em>Copper</em> (<code>C</code>), and <em>Desert</em> (<code>D</code>). They live in a burrow that consists of a <em>hallway</em> and four <em>side rooms</em>. The side rooms are initially full of amphipods, and the hallway is initially empty.</p>
<p>They give you a <em>diagram of the situation</em> (your puzzle input), including locations of each amphipod (<code>A</code>, <code>B</code>, <code>C</code>, or <code>D</code>, each of which is occupying an otherwise open space), walls (<code>#</code>), and open space (<code>.</code>).</p>
<p>For example:</p>
<pre><code>#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
</code></pre>
<p>The amphipods would like a method to organize every amphipod into side rooms so that each side room contains one type of amphipod and the types are sorted <code>A</code>-<code>D</code> going left to right, like this:</p>
<pre><code>#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
</code></pre>
<p>Amphipods can move up, down, left, or right so long as they are moving into an unoccupied open space. Each type of amphipod requires a different amount of <em>energy</em> to move one step: Amber amphipods require <code>1</code> energy per step, Bronze amphipods require <code>10</code> energy, Copper amphipods require <code>100</code>, and Desert ones require <code>1000</code>. The amphipods would like you to find a way to organize the amphipods that requires the <em>least total energy</em>.</p>
<p>However, because they are timid and stubborn, the amphipods have some extra rules:</p>
<ul>
<li>Amphipods will never <em>stop on the space immediately outside any room</em>. They can move into that space so long as they immediately continue moving. (Specifically, this refers to the four open spaces in the hallway that are directly above an amphipod starting position.)</li>
<li>Amphipods will never <em>move from the hallway into a room</em> unless that room is their destination room <em>and</em> that room contains no amphipods which do not also have that room as their own destination. If an amphipod's starting room is not its destination room, it can stay in that room until it leaves the room. (For example, an Amber amphipod will not move from the hallway into the right three rooms, and will only move into the leftmost room if that room is empty or if it only contains other Amber amphipods.)</li>
<li>Once an amphipod stops moving in the hallway, <em>it will stay in that spot until it can move into a room</em>. (That is, once any amphipod starts moving, any other amphipods currently in the hallway are locked in place and will not move again until they can move fully into a room.)</li>
</ul>
<p>In the above example, the amphipods can be organized using a minimum of <code><em>12521</em></code> energy. One way to do this is shown below.</p>
<p>Starting configuration:</p>
<pre><code>#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
</code></pre>
<p>One Bronze amphipod moves into the hallway, taking 4 steps and using <code>40</code> energy:</p>
<pre><code>#############
#...B.......#
###B#C#.#D###
  #A#D#C#A#
  #########
</code></pre>
<p>The only Copper amphipod not in its side room moves there, taking 4 steps and using <code>400</code> energy:</p>
<pre><code>#############
#...B.......#
###B#.#C#D###
  #A#D#C#A#
  #########
</code></pre>
<p>A Desert amphipod moves out of the way, taking 3 steps and using <code>3000</code> energy, and then the Bronze amphipod takes its place, taking 3 steps and using <code>30</code> energy:</p>
<pre><code>#############
#.....D.....#
###B#.#C#D###
  #A#B#C#A#
  #########
</code></pre>
<p>The leftmost Bronze amphipod moves to its room using <code>40</code> energy:</p>
<pre><code>#############
#.....D.....#
###.#B#C#D###
  #A#B#C#A#
  #########
</code></pre>
<p>Both amphipods in the rightmost room move into the hallway, using <code>2003</code> energy in total:</p>
<pre><code>#############
#.....D.D.A.#
###.#B#C#.###
  #A#B#C#.#
  #########
</code></pre>
<p>Both Desert amphipods move into the rightmost room using <code>7000</code> energy:</p>
<pre><code>#############
#.........A.#
###.#B#C#D###
  #A#B#C#D#
  #########
</code></pre>
<p>Finally, the last Amber amphipod moves into its room, using <code>8</code> energy:</p>
<pre><code>#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
</code></pre>
<p><em>What is the least energy required to organize the amphipods?</em></p>
</article>


In [119]:
from copy import deepcopy
from heapq import heappop, heappush
from itertools import chain
from typing import ClassVar


@dataclass(frozen=True, order=True)
class State:
    """
    hallway:  0--1--2--3--4--5--6--7--8--9--10  ↓ i // 2 - 1 if i in {2.4.6.8}
                    |     |     |     |        	↑ 2*(i+1)
    rooms[0]:       0     1     2     3
                    |     |     |     |
    rooms[1]:       0     1     2     3
    """

    no_stopping: ClassVar[tuple[int, int, int, int]] = 2, 4, 6, 8
    destination_for: ClassVar[str] = "ABCD"
    energy_per_step: ClassVar[dict[str, int]] = {"A": 1, "B": 10, "C": 100, "D": 1_000}

    energy: int
    hallway: list[str]
    rooms: list[list[str]]

    def solve(self) -> int:
        heap = [self]
        seen = set()

        while heap:
            state = heappop(heap)

            if state.is_goal():
                return state.energy

            hs = tuple(
                tuple(r) for r in chain(state.hallway, state.rooms[0], state.rooms[1])
            )
            if hs in seen:
                continue

            seen.add(hs)

            state.move_from_hallway_to_room0(heap)
            state.move_from_room0_to_hallway(heap)
            state.move_from_room0_to_room1(heap)
            state.move_from_room1_to_room0(heap)
        return -1

    def is_goal(self) -> bool:
        return all(spot == "." for spot in self.hallway) and all(
            self.rooms[i][r] == self.destination_for[r]
            for i, r in product(range(len(self.rooms)), range(len(self.rooms[0])))
        )

    def move_from_hallway_to_room0(self, heap: list[State]) -> None:
        for i, occupant in enumerate(self.hallway):
            if occupant != ".":
                home_room = self.destination_for.index(occupant)
                if self.rooms[1][home_room] == self.rooms[0][home_room] == "." or (
                    self.rooms[1][home_room] == self.destination_for[home_room]
                    and self.rooms[0][home_room] == "."
                ):
                    to = 2 * (home_room + 1)
                    if to >= i:
                        current = i + 1
                        delta = 1
                    else:
                        current = i - 1
                        delta = -1

                    while (
                        0 <= current < len(self.hallway)
                        and self.hallway[current] == "."
                        and current != to
                    ):
                        current += delta

                    if current == to:
                        energy = (abs(to - i) + 1) * self.energy_per_step[occupant]
                        hallway = self.hallway[:]
                        rooms = deepcopy(self.rooms)
                        hallway[i], rooms[0][home_room] = (
                            rooms[0][home_room],
                            hallway[i],
                        )
                        energy = (
                            self.energy
                            + (abs(i - to) + 1) * self.energy_per_step[occupant]
                        )
                        heappush(heap, State(energy, hallway, rooms))

    def move_from_room0_to_room1(self, heap: list[State]) -> None:
        for i, occupant in enumerate(self.rooms[0]):
            if occupant == self.destination_for[i] and self.rooms[1][i] == ".":
                hallway = self.hallway[:]
                rooms = deepcopy(self.rooms)
                rooms[1][i], rooms[0][i] = rooms[0][i], rooms[1][i]
                energy = self.energy + self.energy_per_step[occupant]
                heappush(heap, State(energy, hallway, rooms))

    def move_from_room1_to_room0(self, heap: list[State]) -> None:
        for i, occupant in enumerate(self.rooms[1]):
            if (
                occupant not in (".", self.destination_for[i])
                and self.rooms[0][i] == "."
            ):
                hallway = self.hallway[:]
                rooms = deepcopy(self.rooms)
                rooms[1][i], rooms[0][i] = rooms[0][i], rooms[1][i]
                energy = self.energy + self.energy_per_step[occupant]
                heappush(heap, State(energy, hallway, rooms))

    def move_from_room0_to_hallway(self, heap: list[State]) -> None:
        for i, occupier in enumerate(self.rooms[0]):
            hallway_i = 2 * (i + 1)
            if occupier != "." and not (
                occupier == self.rooms[1][i] == self.destination_for[i]
            ):
                # move left
                steps = 2
                next_left = hallway_i - 1
                while next_left > -1 and self.hallway[next_left] == ".":
                    if next_left not in self.no_stopping:
                        hallway = self.hallway[:]
                        rooms = deepcopy(self.rooms)
                        hallway[next_left], rooms[0][i] = (
                            rooms[0][i],
                            hallway[next_left],
                        )
                        energy = self.energy + steps * self.energy_per_step[occupier]
                        heappush(heap, State(energy, hallway, rooms))
                    next_left -= 1
                    steps += 1

                # move right
                steps = 2
                next_right = hallway_i + 1
                while (
                    next_right < len(self.hallway) and self.hallway[next_right] == "."
                ):
                    if next_right not in self.no_stopping:
                        hallway = self.hallway[:]
                        rooms = deepcopy(self.rooms)
                        hallway[next_right], rooms[0][i] = (
                            rooms[0][i],
                            hallway[next_right],
                        )
                        energy = self.energy + steps * self.energy_per_step[occupier]
                        heappush(heap, State(energy, hallway, rooms))
                    next_right += 1
                    steps += 1

    def __str__(self) -> str:
        return "\n".join(
            (
                "#" * 13,
                f"#{''.join(spot for spot in self.hallway)}#",
                f"###{'#'.join(spot for spot in self.rooms[0])}###",
                f"  #{'#'.join(spot for spot in self.rooms[1])}#",
                f"  {'#'*9}",
            )
        )

    @classmethod
    def create_from_diagram(cls, diagram: str, energy: int = 0) -> State:
        data = list(re.sub(r"[^A-D^\.]", "", diagram))
        hallway = data[:11]
        rooms = [data[11:15], data[15:]]
        return State(energy, hallway, rooms)

In [120]:
solve_tests = [
    {
        "name": "initial is goal",
        "diagram": """
           #############
           #...........#
           ###A#B#C#D###
             #A#B#C#D#
             #########""",
        "expected": 0,
    },
    {
        "name": "penultimum from example",
        "diagram": """
          #############
          #.........A.#
          ###.#B#C#D###
            #A#B#C#D#
            ##########""",
        "expected": 8,
    },
    {
        "name": "2 before goal from example",
        "diagram": """
            #############
            #.....D.D.A.#
            ###.#B#C#.###
              #A#B#C#.#
              #########""",
        "expected": 7008,
    },
    {
        "name": "3 before goal from example",
        "diagram": """
            #############
            #.....D.....#
            ###.#B#C#D###
              #A#B#C#A#
              ######### 
            """,
        "expected": 9011,
    },
    {
        "name": "4 before goal from example",
        "diagram": """
            #############
            #.....D.....#
            ###B#.#C#D###
              #A#B#C#A#
              #########            """,
        "expected": 9051,
    },
    {
        "name": "5 before goal from example",
        "diagram": """
           #############
           #...B.......#
           ###B#.#C#D###
             #A#D#C#A#
             #########""",
        "expected": 12081,
    },
    {
        "name": "6 before goal from example",
        "diagram": """
            #############
            #...B.......#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
        "expected": 12481,
    },
    {
        "name": "example",
        "diagram": """
            #############
            #...........#
            ###B#C#B#D###
              #A#D#C#A#
              #########""",
        "expected": 12521,
    },
]


@test(tests=solve_tests)
def solve_test(diagram: str) -> bool:
    state = State.create_from_diagram(diagram)
    return state.solve()


Test initial is goal passed, for solve_test.
Test penultimum from example passed, for solve_test.
Test 2 before goal from example passed, for solve_test.
Test 3 before goal from example passed, for solve_test.
Test 4 before goal from example passed, for solve_test.
Test 5 before goal from example passed, for solve_test.
Test 6 before goal from example passed, for solve_test.
Test example passed, for solve_test.
Success


In [121]:
is_goal_tests = [
    {
        "name": "is goal",
        "diagram": """
           #############
           #...........#
           ###A#B#C#D###
             #A#B#C#D#
             #########""",
        "expected": True,
    },
    {
        "name": "is not goal",
        "diagram": """
           #############
           #...........#
           ###A#B#C#D###
             #A#B#D#C#
             #########""",
        "expected": False,
    },
    {
        "name": "Is not goal",
        "diagram": """
           #############
           #.....D.....#
           ###.#.#C#.###
             #A#B#C#D#
             #########""",
        "expected": False,
    },
]


@test(tests=is_goal_tests)
def is_goal_testzs(diagram: str) -> bool:
    state = State.create_from_diagram(diagram)
    return state.is_goal()


Test is goal passed, for is_goal_testzs.
Test is not goal passed, for is_goal_testzs.
Test Is not goal passed, for is_goal_testzs.
Success


In [122]:
def assert_function(actual: list[State], expected: list[str]) -> bool:
    if len(actual) != len(expected):
        return False

    for a, e in zip(
        sorted(actual), sorted(State.create_from_diagram(d, c) for d, c in expected)
    ):
        if a != e:
            print("**Error:")
            print(a)
            print(a.energy)
            print("\nshould be:\n")
            print(e)
            print(e.energy)
            return False
    return True

In [123]:
move_from_hallway_to_room0_tests = [
    {
        "name": "Move none from hallway to room[1] test 1",
        "diagram": """
           #############
           #.....D.....#
           ###.#.#C#D###
             #A#B#C#A#
             #########""",
        "expected": [],
    },
    {
        "name": "Move one from room[0] to room[1] test 2",
        "diagram": """
           #############
           #.....D.....#
           ###.#.#C#.###
             #A#B#C#D#
             #########""",
        "expected": [
            (
                """
               #############
               #...........#
               ###.#.#C#D###
                 #A#B#C#D#
                 #########""",
                4 * State.energy_per_step["D"],
            ),
        ],
    },
    {
        "name": "Move one from room[0] to room[1] test 3",
        "diagram": """
           #############
           #.....A.....#
           ###.#.#C#.###
             #.#B#C#D#
             #########""",
        "expected": [
            (
                """
               #############
               #...........#
               ###A#.#C#.###
                 #.#B#C#D#
                 #########""",
                4 * State.energy_per_step["A"],
            ),
        ],
    },
    {
        "name": "Move one from room[0] to room[1] test 4",
        "diagram": """
           #############
           #.....A.....#
           ###.#.#C#.###
             #B#B#C#D#
             #########""",
        "expected": [],
    },
]


@test(tests=move_from_hallway_to_room0_tests, assert_funct=assert_function)
def move_from_hallway_to_room0_test(diagram: str) -> list[State]:
    state = State.create_from_diagram(diagram)
    heap = []
    state.move_from_hallway_to_room0(heap)
    return heap


Test Move none from hallway to room[1] test 1 passed, for move_from_hallway_to_room0_test.
Test Move one from room[0] to room[1] test 2 passed, for move_from_hallway_to_room0_test.
Test Move one from room[0] to room[1] test 3 passed, for move_from_hallway_to_room0_test.
Test Move one from room[0] to room[1] test 4 passed, for move_from_hallway_to_room0_test.
Success


In [124]:
move_from_room0_to_room1_tests = [
    {
        "name": "Move none from room[0] to room[1] test 1",
        "diagram": """
            #############
            #...........#
            ###B#C#B#.###
              #.#.#.#A#
              #########""",
        "expected": [],
    },
    {
        "name": "Move one from room[0] to room[1] test 1",
        "diagram": """
            #############
            #...........#
            ###.#C#.#D###
              #A#D#C#.#
              #########""",
        "expected": [
            (
                """
                #############
                #...........#
                ###.#C#.#.###
                  #A#D#C#D#
                  #########""",
                State.energy_per_step["D"],
            ),
        ],
    },
]


@test(tests=move_from_room0_to_room1_tests, assert_funct=assert_function)
def move_from_room0_to_room1_test(diagram: str) -> list[State]:
    state = State.create_from_diagram(diagram)
    heap = []
    state.move_from_room0_to_room1(heap)
    return heap


Test Move none from room[0] to room[1] test 1 passed, for move_from_room0_to_room1_test.
Test Move one from room[0] to room[1] test 1 passed, for move_from_room0_to_room1_test.
Success


In [125]:
move_from_room1_to_room0_tests = [
    {
        "name": "Move none from room[1] to room[0] test 1",
        "diagram": """
            #############
            #...........#
            ###B#C#B#D###
              #A#D#C#A#
              #########""",
        "expected": [],
    },
    {
        "name": "Move one from room[1] to room[0] test 1",
        "diagram": """
            #############
            #...........#
            ###.#C#.#.###
              #A#D#C#A#
              #########""",
        "expected": [
            (
                """
                #############
                #...........#
                ###.#C#.#A###
                  #A#D#C#.#
                  #########""",
                State.energy_per_step["A"],
            ),
        ],
    },
]


@test(tests=move_from_room1_to_room0_tests, assert_funct=assert_function)
def move_from_room1_to_room0_test(diagram: str) -> list[State]:
    state = State.create_from_diagram(diagram)
    heap = []
    state.move_from_room1_to_room0(heap)
    return heap


Test Move none from room[1] to room[0] test 1 passed, for move_from_room1_to_room0_test.
Test Move one from room[1] to room[0] test 1 passed, for move_from_room1_to_room0_test.
Success


In [126]:
move_from_room0_to_hallway_tests = [
    {
        "name": "move_from_room0_to_hallway on initial diagram test 1",
        "diagram": """
            #############
            #...........#
            ###B#.#.#.###
              #A#.#.#.#
              #########""",
        "expected": [
            # A room[0]
            (
                """
              #############
              #B..........#
              ###.#.#.#.###
                #A#.#.#.#
                #########""",
                3 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #.B.........#
              ###.#.#.#.###
                #A#.#.#.#
                #########""",
                2 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #...B.......#
              ###.#.#.#.###
                #A#.#.#.#
                #########""",
                2 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #.....B.....#
              ###.#.#.#.###
                #A#.#.#.#
                #########""",
                4 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #.......B...#
              ###.#.#.#.###
                #A#.#.#.#
                #########""",
                6 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #.........B.#
              ###.#.#.#.###
                #A#.#.#.#
                #########""",
                8 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #..........B#
              ###.#.#.#.###
                #A#.#.#.#
                #########""",
                9 * State.energy_per_step["B"],
            ),
        ],
    },
    {
        "name": "move_from_room0_to_hallway on initial diagram test 2",
        "diagram": """
            #############
            #...........#
            ###B#C#B#D###
              #A#D#C#A#
              #########""",
        "expected": [
            # A room[0]
            (
                """
              #############
              #B..........#
              ###.#C#B#D###
                #A#D#C#A#
                #########""",
                3 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #.B.........#
              ###.#C#B#D###
                #A#D#C#A#
                #########""",
                2 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #...B.......#
              ###.#C#B#D###
                #A#D#C#A#
                #########""",
                2 * State.energy_per_step["B"],
            ),
            (
                """
              #############
              #.....B.....#
              ###.#C#B#D###
                #A#D#C#A#
                #########""",
                4 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #.......B...#
            ###.#C#B#D###
              #A#D#C#A#
              #########""",
                6 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #.........B.#
            ###.#C#B#D###
              #A#D#C#A#
              #########""",
                8 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #..........B#
            ###.#C#B#D###
              #A#D#C#A#
              #########""",
                9 * State.energy_per_step["B"],
            ),
            # B room[0]
            (
                """
            #############
            #C..........#
            ###B#.#B#D###
              #A#D#C#A#
              #########""",
                5 * State.energy_per_step["C"],
            ),
            (
                """
            #############
            #.C.........#
            ###B#.#B#D###
              #A#D#C#A#
              #########""",
                4 * State.energy_per_step["C"],
            ),
            (
                """
            #############
            #...C.......#
            ###B#.#B#D###
              #A#D#C#A#
              #########""",
                2 * State.energy_per_step["C"],
            ),
            (
                """
            #############
            #.....C.....#
            ###B#.#B#D###
              #A#D#C#A#
              #########""",
                2 * State.energy_per_step["C"],
            ),
            (
                """
            #############
            #.......C...#
            ###B#.#B#D###
              #A#D#C#A#
              #########""",
                4 * State.energy_per_step["C"],
            ),
            (
                """
            #############
            #.........C.#
            ###B#.#B#D###
              #A#D#C#A#
              #########""",
                6 * State.energy_per_step["C"],
            ),
            (
                """
            #############
            #..........C#
            ###B#.#B#D###
              #A#D#C#A#
              #########""",
                7 * State.energy_per_step["C"],
            ),
            # C room[0]
            (
                """
            #############
            #B..........#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
                7 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #.B.........#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
                6 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #...B.......#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
                4 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #.....B.....#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
                2 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #.......B...#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
                2 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #.........B.#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
                4 * State.energy_per_step["B"],
            ),
            (
                """
            #############
            #..........B#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
                5 * State.energy_per_step["B"],
            ),
            # D room[0]
            (
                """
            #############
            #D..........#
            ###B#C#B#.###
              #A#D#C#A#
              #########""",
                9 * State.energy_per_step["D"],
            ),
            (
                """
            #############
            #.D.........#
            ###B#C#B#.###
              #A#D#C#A#
              #########""",
                8 * State.energy_per_step["D"],
            ),
            (
                """
            #############
            #...D.......#
            ###B#C#B#.###
              #A#D#C#A#
              #########""",
                6 * State.energy_per_step["D"],
            ),
            (
                """
            #############
            #.....D.....#
            ###B#C#B#.###
              #A#D#C#A#
              #########""",
                4 * State.energy_per_step["D"],
            ),
            (
                """
            #############
            #.......D...#
            ###B#C#B#.###
              #A#D#C#A#
              #########""",
                2 * State.energy_per_step["D"],
            ),
            (
                """
            #############
            #.........D.#
            ###B#C#B#.###
              #A#D#C#A#
              #########""",
                2 * State.energy_per_step["D"],
            ),
            (
                """
            #############
            #..........D#
            ###B#C#B#.###
              #A#D#C#A#
              #########""",
                3 * State.energy_per_step["D"],
            ),
        ],
    },
    {
        "name": "No moves from room[0] to hallway test 3",
        "diagram": """
            #############
            #.....D.D.A.#
            ###.#B#C#.###
              #A#B#C#.#
              #########""",
        "expected": [],
    },
    {
        "name": "Moves going to right are blocked from room[0] to hallway test 4",
        "diagram": """
            #############
            #.....D.....#
            ###.#A#C#.###
              #A#B#C#.#
              #########""",
        "expected": [
            (
                """
              #############
              #...A.D.....#
              ###.#.#C#.###
                #A#B#C#.#
                #########""",
                2 * State.energy_per_step["A"],
            ),
            (
                """
              #############
              #.A...D.....#
              ###.#.#C#.###
                #A#B#C#.#
                #########""",
                4 * State.energy_per_step["A"],
            ),
            (
                """
              #############
              #A....D.....#
              ###.#.#C#.###
                #A#B#C#.#
                #########""",
                5 * State.energy_per_step["A"],
            ),
        ],
    },
    {
        "name": "Moves going to left are blocked from room[0] to hallway test 5",
        "diagram": """
            #############
            #...D.......#
            ###.#A#C#.###
              #A#B#C#.#
              #########""",
        "expected": [
            (
                """
              #############
              #...D.A.....#
              ###.#.#C#.###
                #A#B#C#.#
                #########""",
                2 * State.energy_per_step["A"],
            ),
            (
                """
              #############
              #...D...A...#
              ###.#.#C#.###
                #A#B#C#.#
                #########""",
                4 * State.energy_per_step["A"],
            ),
            (
                """
              #############
              #...D.....A.#
              ###.#.#C#.###
                #A#B#C#.#
                #########""",
                6 * State.energy_per_step["A"],
            ),
            (
                """
              #############
              #...D......A#
              ###.#.#C#.###
                #A#B#C#.#
                #########""",
                7 * State.energy_per_step["A"],
            ),
        ],
    },
]


@test(tests=move_from_room0_to_hallway_tests, assert_funct=assert_function)
def move_from_room0_to_hallway_test(diagram: str) -> list[State]:
    state = State.create_from_diagram(diagram)
    heap = []
    state.move_from_room0_to_hallway(heap)
    return heap


Test move_from_room0_to_hallway on initial diagram test 1 passed, for move_from_room0_to_hallway_test.
Test move_from_room0_to_hallway on initial diagram test 2 passed, for move_from_room0_to_hallway_test.
Test No moves from room[0] to hallway test 3 passed, for move_from_room0_to_hallway_test.
Test Moves going to right are blocked from room[0] to hallway test 4 passed, for move_from_room0_to_hallway_test.
Test Moves going to left are blocked from room[0] to hallway test 5 passed, for move_from_room0_to_hallway_test.
Success


In [127]:
constructor_tests = [
    {
        "name": "Constructor test 1",
        "diagram": """
            #############
            #...........#
            ###B#C#B#D###
              #A#D#C#A#
              #########""",
        "expected": State(
            0,
            [".", ".", ".", ".", ".", ".", ".", ".", ".", ".", "."],
            [
                ["B", "C", "B", "D"],
                ["A", "D", "C", "A"],
            ],
        ),
    },
    {
        "name": "Constructor test 2",
        "diagram": """
            #############
            #.....D.D.A.#
            ###.#B#C#.###
              #A#B#C#.#
              #########""",
        "expected": State(
            0,
            [".", ".", ".", ".", ".", "D", ".", "D", ".", "A", "."],
            [
                [".", "B", "C", "."],
                ["A", "B", "C", "."],
            ],
        ),
    },
]


@test(tests=constructor_tests)
def constructor_test(diagram: str) -> State:
    return State.create_from_diagram(diagram)


Test Constructor test 1 passed, for constructor_test.
Test Constructor test 2 passed, for constructor_test.
Success


In [128]:
example = """
#############
#...........#
###B#C#B#D###
  #A#D#C#A#      
  #########    
"""

# assert State.create_from_diagram(example).solve() == 12521

In [129]:
puzzle = """
#############
#...........#
###D#D#C#C###
  #B#A#B#A#  
  #########  
"""
# print(f"Part I: { State.create_from_diagram(puzzle).solve() }")

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>16059</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As you prepare to give the amphipods your solution, you notice that the diagram they handed you was actually folded up. As you unfold it, you discover an extra part of the diagram.</p>
<p>Between the first and second lines of text that contain amphipod starting positions, insert the following lines:</p>
<pre><code>  #D#C#B#A#
  #D#B#A#C#
</code></pre>
<p>So, the above example now becomes:</p>
<pre><code>#############
#...........#
###B#C#B#D###
  <em>#D#C#B#A#
  #D#B#A#C#</em>
  #A#D#C#A#
  #########
</code></pre>
<p>The amphipods still want to be organized into rooms similar to before:</p>
<pre><code>#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########
</code></pre>
<p>In this updated example, the least energy required to organize these amphipods is <code><em>44169</em></code>:</p>
<pre><code>
#############
#...........#
###B#C#B#D###
  #D#C#B#A#
  #D#B#A#C#
  #A#D#C#A#
  #########
</code></pre>
<pre><code>
#############
#..........D#
###B#C#B#.###
  #D#C#B#A#
  #D#B#A#C#
  #A#D#C#A#
  #########
</code></pre>
<pre><code>
#############
#A.........D#
###B#C#B#.###
  #D#C#B#.#
  #D#B#A#C#
  #A#D#C#A#
  #########
</code></pre>
<pre><code>
#############
#A........BD#
###B#C#.#.###
  #D#C#B#.#
  #D#B#A#C#
  #A#D#C#A#
  #########
</code></pre>
<pre><code>
#############
#A......B.BD#
###B#C#.#.###
  #D#C#.#.#
  #D#B#A#C#
  #A#D#C#A#
  #########
</code></pre>
<pre><code>
#############
#AA.....B.BD#
###B#C#.#.###
  #D#C#.#.#
  #D#B#.#C#
  #A#D#C#A#
  #########
</code></pre>
<pre><code>
#############
#AA.....B.BD#
###B#.#.#.###
  #D#C#.#.#
  #D#B#C#C#
  #A#D#C#A#
  #########
</code></pre>
<pre><code>
#############
#AA.....B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#B#C#C#
  #A#D#C#A#
  #########
</code></pre>
<pre><code>
#############
#AA...B.B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#.#C#C#
  #A#D#C#A#
  #########
</code></pre>
<pre><code>
#############
#AA.D.B.B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#.#C#C#
  #A#.#C#A#
  #########
</code></pre>
<pre><code>
#############
#AA.D...B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#.#C#C#
  #A#B#C#A#
  #########
</code></pre>
<pre><code>
#############
#AA.D.....BD#
###B#.#.#.###
  #D#.#C#.#
  #D#B#C#C#
  #A#B#C#A#
  #########
</code></pre>
<pre><code>
#############
#AA.D......D#
###B#.#.#.###
  #D#B#C#.#
  #D#B#C#C#
  #A#B#C#A#
  #########
</code></pre>
<pre><code>
#############
#AA.D......D#
###B#.#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#A#
  #########
</code></pre>
<pre><code>
#############
#AA.D.....AD#
###B#.#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#.#
  #########
</code></pre>
<pre><code>
#############
#AA.......AD#
###B#.#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#D#
  #########
</code></pre>
<pre><code>
#############
#AA.......AD#
###.#B#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#D#
  #########
</code></pre>
<pre><code>
#############
#AA.......AD#
###.#B#C#.###
  #.#B#C#.#
  #D#B#C#D#
  #A#B#C#D#
  #########
</code></pre>
<pre><code>
#############
#AA.D.....AD#
###.#B#C#.###
  #.#B#C#.#
  #.#B#C#D#
  #A#B#C#D#
  #########
  </code></pre>
<pre><code>
#############
#A..D.....AD#
###.#B#C#.###
  #.#B#C#.#
  #A#B#C#D#
  #A#B#C#D#
  #########
  </code></pre>
<pre><code>
#############
#...D.....AD#
###.#B#C#.###
  #A#B#C#.#
  #A#B#C#D#
  #A#B#C#D#
  #########
</code></pre>
<pre><code>
#############
#.........AD#
###.#B#C#.###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########
</code></pre>
<pre><code>
#############
#..........D#
###A#B#C#.###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########
</code></pre>
<pre><code>
#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########
</code></pre>

<p>Using the initial configuration from the full diagram, <em>what is the least energy required to organize the amphipods?</em></p>
</article>


<link href="style.css" rel="stylesheet"></link>


In [130]:
from enum import IntEnum


class Amphipod(IntEnum):
    __energy_per_step = [1, 10, 100, 1_000]

    A = 0
    B = 1
    C = 2
    D = 3
    E = 4

    def empty(self) -> bool:
        return self.name == Amphipod.E

    def energy_per_step(self) -> int:
        return self.__energy_per_step[self.value]

    @classmethod
    def chr(cls, s: str) -> Amphipod:
        if s == ".":
            return Amphipod.E
        return cls.__getitem__(s)

    def __str__(self) -> str:
        return f"{'.' if self == Amphipod.E else self.name}"

    def __repr__(self) -> str:
        return str(self)


A, B, C, D, E = Amphipod.A, Amphipod.B, Amphipod.C, Amphipod.D, Amphipod.E

In [131]:
type Hallway = tuple[
    Amphipod,
    Amphipod,
    Amphipod,
    Amphipod,
    Amphipod,
    Amphipod,
    Amphipod,
    Amphipod,
    Amphipod,
    Amphipod,
    Amphipod,
]
type Room = tuple[Amphipod, ...]
type Rooms = tuple[Room, Room, Room, Room]

In [132]:
@dataclass(frozen=True, order=True)
class StateII:
    """
    hallway:  0--1--2--3--4--5--6--7--8--9--10  ↓ i // 2 - 1 if i in {2.4.6.8}
                    |     |     |     |        	↑ 2*(i+1)
    rooms[0]:       0     1     2     3
                    |     |     |     |
    rooms[1]:       0     1     2     3
                    |     |     |     |
    rooms[2]:       0     1     2     3
                    |     |     |     |
    rooms[3]:       0     1     2     3
    """

    no_stopping: ClassVar[tuple[int, int, int, int]] = 2, 4, 6, 8

    energy: int
    movable: tuple[tuple[Amphipod, int, int], ...]  # (Amphipod, room/hallway,spot)
    hallway: Hallway
    rooms: Rooms
    n: int

    def solve(self) -> int:
        heap: list[StateII] = [self]
        seen = set()

        while heap:
            state = heappop(heap)

            if state.is_goal():
                return state.energy

            st = state.hallway, state.rooms

            if st in seen:
                continue

            seen.add(st)

            for i_movable, (amp, room, spot) in enumerate(state.movable):
                if room == -1:  # Hallway
                    if state.can_move(amp, spot):
                        heappush(
                            heap,
                            state.from_hallway_to_room(i_movable),
                        )
                elif state.can_move(amp, state.no_stopping[room]):
                    heappush(heap, state.from_room_to_room(i_movable))
                else:
                    start = state.no_stopping[room]
                    # left
                    left = start
                    while 0 <= left and state.hallway[left] == E:
                        if left not in state.no_stopping:
                            heappush(
                                heap,
                                state.move_from_room_to_hallway(i_movable, left),
                            )
                        left -= 1
                    # right
                    right = start
                    while right < len(state.hallway) and state.hallway[right] == E:
                        if right not in state.no_stopping:
                            heappush(
                                heap,
                                state.move_from_room_to_hallway(i_movable, right),
                            )
                        right += 1

        return -1

    def can_move(self, amp, spot) -> bool:
        return self.room_can_receive(amp) and self.hallway_is_free(
            spot, self.no_stopping[amp.value]
        )

    def hallway_is_free(self, fr: int, to: int) -> bool:
        if fr < to:
            fr += 1
        else:
            fr, to = to, fr

        return all(self.hallway[i] == E for i in range(fr, to))

    def room_can_receive(self, amp: Amphipod) -> bool:
        return all(amp == E for amp in self.rooms[amp.value])

    def from_room_to_room(self, i_movable: int) -> StateII:
        amphipod, room, spot = self.movable[i_movable]

        energy = self.energy + amphipod.energy_per_step() * (
            spot
            + 1
            + abs(self.no_stopping[room] - self.no_stopping[amphipod.value])
            + len(self.rooms[amphipod.value])
        )
        movable = list(self.movable)
        movable.pop(i_movable)

        rooms = list(self.rooms)

        rooms[room] = list(rooms[room])
        rooms[room][spot] = E
        rooms[room] = tuple(rooms[room])

        if spot < len(rooms[room]) - 1:
            movable.append((rooms[room][spot + 1], room, spot + 1))

        movable = tuple(movable)

        rooms[amphipod.value] = list(rooms[amphipod.value])
        rooms[amphipod.value].pop()
        rooms[amphipod.value] = tuple(rooms[amphipod.value])

        rooms = tuple(rooms)

        return StateII(energy, movable, self.hallway, rooms, self.n)

    def from_hallway_to_room(self, i_movable: int) -> StateII:
        amphipod, _, spot = self.movable[i_movable]

        energy = self.energy + amphipod.energy_per_step() * (
            abs(spot - self.no_stopping[amphipod.value])
            + len(self.rooms[amphipod.value])
        )

        movable = list(self.movable)
        movable.pop(i_movable)
        movable = tuple(movable)

        hallway = list(self.hallway)
        hallway[spot] = E
        hallway = tuple(hallway)

        rooms = list(self.rooms)
        rooms[amphipod.value] = list(rooms[amphipod.value])
        rooms[amphipod.value].pop()
        rooms[amphipod.value] = tuple(rooms[amphipod.value])
        rooms = tuple(rooms)

        return StateII(energy, movable, hallway, rooms, self.n)

    def move_from_room_to_hallway(self, i_movable: int, hallway_spot: int) -> StateII:
        amphipod, room, spot = self.movable[i_movable]

        energy = self.energy + amphipod.energy_per_step() * (
            spot + 1 + abs(hallway_spot - self.no_stopping[room])
        )

        movable = list(self.movable)
        movable.pop(i_movable)
        movable.append((amphipod, -1, hallway_spot))

        hallway = list(self.hallway)
        hallway[hallway_spot] = amphipod
        hallway = tuple(hallway)

        rooms = list(self.rooms)
        rooms[room] = list(rooms[room])
        rooms[room][spot] = E

        if spot < len(rooms[room]) - 1:
            movable.append((rooms[room][spot + 1], room, spot + 1))

        rooms[room] = tuple(rooms[room])
        rooms = tuple(rooms)

        movable = tuple(movable)

        return StateII(energy, movable, hallway, rooms, self.n)

    def is_goal(self) -> bool:
        return all(not room for room in self.rooms)

    def __str__(self) -> str:
        rooms, spots = 4, self.n
        rs = "ABCD"
        return "\n".join(
            (
                f"State(",
                f"\tenergy={self.energy}",
                "",
                f"\t{'#'*13}",
                f'\t#{"".join(f'{a}' for a in self.hallway)}#',
                f"\t###{'#'.join(str(self.rooms[r][0] if self.rooms[r] else f'{rs[r]}')for r in range(rooms))}###",
                *[
                    f"\t  #{'#'.join(str(self.rooms[r][s] if s < len(self.rooms[r]) else f'{rs[r]}')for r in range(rooms))}#"
                    for s in range(1, spots)
                ],
                f"\t  {'#'*9}",
                ")",
            )
        )

    @classmethod
    def parse(cls, diagram: str, cost: int = 0) -> StateII:
        temp = [
            list(re.sub(r"#", "", l.strip()))
            for l in diagram.strip().splitlines()[1:-1]
        ]
        hallway = tuple(Amphipod.chr(c) for c in temp.pop(0))
        rooms = [[Amphipod.chr(c) for c in col] for col in zip(*temp)]
        n = len(rooms[0])

        for i, room in enumerate(rooms):
            while room and room[-1].value == i:
                room.pop()

        rooms = tuple(tuple(room) for room in rooms)
        movables = tuple(
            cls.get_spot(room, r)
            for r, room in enumerate(rooms)
            if room and any(amp != E for amp in room)
        ) + tuple((amp, -1, spot) for spot, amp in enumerate(hallway) if amp != E)
        return cls(0, movables, hallway, rooms, n)

    @staticmethod
    def get_spot(room: list[Amphipod], room_index: int) -> tuple[Amphipod, int, int]:
        for spot, amphipod in enumerate(room):
            if amphipod != E:
                return amphipod, room_index, spot


tests_part_II = [
    {
        "name": "initial is goal",
        "diagram": """
           #############
           #...........#
           ###A#B#C#D###
             #A#B#C#D#
             #########""",
        "expected": 0,
    },
    {
        "name": "penultimum from example",
        "diagram": """
          #############
          #.........A.#
          ###.#B#C#D###
            #A#B#C#D#
            ##########""",
        "expected": 8,
    },
    {
        "name": "2 before goal from example",
        "diagram": """
            #############
            #.....D.D.A.#
            ###.#B#C#.###
              #A#B#C#.#
              #########""",
        "expected": 7008,
    },
    {
        "name": "3 before goal from example",
        "diagram": """
            #############
            #.....D.....#
            ###.#B#C#D###
              #A#B#C#A#
              ######### 
            """,
        "expected": 9011,
    },
    {
        "name": "4 before goal from example",
        "diagram": """
            #############
            #.....D.....#
            ###B#.#C#D###
              #A#B#C#A#
              #########            """,
        "expected": 9051,
    },
    {
        "name": "5 before goal from example",
        "diagram": """
           #############
           #...B.......#
           ###B#.#C#D###
             #A#D#C#A#
             #########""",
        "expected": 12081,
    },
    {
        "name": "6 before goal from example",
        "diagram": """
            #############
            #...B.......#
            ###B#C#.#D###
              #A#D#C#A#
              #########""",
        "expected": 12481,
    },
    {
        "name": "example",
        "diagram": """
            #############
            #...........#
            ###B#C#B#D###
              #A#D#C#A#
              #########""",
        "expected": 12521,
    },
    {
        "name": "example part I",
        "diagram": f"""
                 #############
                 #...........#
                 ###B#C#B#D###
                   #A#D#C#A#      
                   #########    
                 """,
        "expected": 12521,
    },
    {
        "name": "example puzzle part I",
        "diagram": f"""
                   #############
                   #...........#
                   ###D#D#C#C###
                     #B#A#B#A#
                     #########""",
        "expected": 16059,
    },
    {
        "name": "example part II",
        "diagram": f"""
                 #############
                 #...........#
                 ###B#C#B#D###
                   #D#C#B#A#
                   #D#B#A#C#
                   #A#D#C#A#      
                   #########    
                 """,
        "expected": 44169,
    },
]


@test(tests=tests_part_II)
def test_part_II(diagram: str) -> int:
    return StateII.parse(diagram).solve()


Test initial is goal passed, for test_part_II.
Test penultimum from example passed, for test_part_II.
Test 2 before goal from example passed, for test_part_II.
Test 3 before goal from example passed, for test_part_II.
Test 4 before goal from example passed, for test_part_II.
Test 5 before goal from example passed, for test_part_II.
Test 6 before goal from example passed, for test_part_II.
Test example passed, for test_part_II.
Test example part I passed, for test_part_II.
Test example puzzle part I passed, for test_part_II.
Test example part II passed, for test_part_II.
Success


In [133]:
tests_from_room_to_hallway = [
    {
        "name": "Part I Test Room to hallway to the left",
        "diagram": f"""
                 #############
                 #B........DD#
                 ###C#.#B#.###
                   #A#.#C#A#      
                   #########    
                 """,
        "i_movable": 1,
        "hallway_spot": 1,
        "expected": StateII(
            energy=60,
            movable=(
                (C, 0, 0),
                (A, 3, 1),
                (B, -1, 0),
                (D, -1, 9),
                (D, -1, 10),
                (B, -1, 1),
            ),
            hallway=tuple({0: B, 1: B, 9: D, 10: D}.get(i, E) for i in range(11)),
            rooms=(
                (C,),
                (E, E),
                (E,),
                (E, A),
            ),
            n=2,
        ),
    },
    {
        "name": "Part I Test Room to hallway to the left",
        "diagram": f"""
                 #############
                 #BD.D.......#
                 ###C#.#B#.###
                   #A#.#C#A#      
                   #########    
                 """,
        "i_movable": 1,
        "hallway_spot": 10,
        "expected": StateII(
            energy=50,
            movable=(
                (C, 0, 0),
                (A, 3, 1),
                (B, -1, 0),
                (D, -1, 1),
                (D, -1, 3),
                (B, -1, 10),
            ),
            hallway=tuple({0: B, 1: D, 3: D, 10: B}.get(i, E) for i in range(11)),
            rooms=(
                (C,),
                (E, E),
                (E,),
                (E, A),
            ),
            n=2,
        ),
    },
    {
        "name": "Part II test 1",
        "diagram": f"""
                  #############
                  #AA.D......D#
                  ###B#.#C#.###
                    #D#B#C#.#
                    #D#B#C#.#
                    #A#B#C#A#
                    #########
                 """,
        "i_movable": 1,
        "hallway_spot": 9,
        "expected": StateII(
            energy=5,
            movable=(
                (B, 0, 0),
                (A, -1, 0),
                (A, -1, 1),
                (D, -1, 3),
                (D, -1, 10),
                (A, -1, 9),
            ),
            hallway=tuple({0: A, 1: A, 3: D, 9: A, 10: D}.get(i, E) for i in range(11)),
            rooms=(
                (B, D, D),
                (E,),
                tuple(),
                (E, E, E, E),
            ),
            n=4,
        ),
    },
    {
        "name": "Part II test 2",
        "diagram": f"""
                  #############
                  #AA.......DD#
                  ###B#.#C#.###
                    #D#B#C#.#
                    #D#B#C#.#
                    #A#B#C#A#
                    #########
                 """,
        "i_movable": 0,
        "hallway_spot": 3,
        "expected": StateII(
            energy=20,
            movable=(
                (A, 3, 3),
                (A, -1, 0),
                (A, -1, 1),
                (D, -1, 9),
                (D, -1, 10),
                (B, -1, 3),
                (D, 0, 1),
            ),
            hallway=tuple({0: A, 1: A, 3: B, 9: D, 10: D}.get(i, E) for i in range(11)),
            rooms=(
                (E, D, D),
                (E,),
                tuple(),
                (E, E, E, A),
            ),
            n=4,
        ),
    },
]

PRT = False


def assert_function(actual: StateII, expected: StateII) -> bool:
    if PRT:
        print(f"Energy: actual={actual.energy} and expected={expected.energy}")
        print(f"Moveable:")
        print(f"actual  ={actual.movable}")
        print(f"expected={expected.movable}")
        print(f"Hallway:")
        print(f"actual  ={actual.hallway}")
        print(f"expected={expected.hallway}")
        print(f"Rooms:")
        print(f"actual  ={actual.rooms}")
        print(f"expected={expected.rooms}")

    return actual == expected


@test(tests=tests_from_room_to_hallway, assert_funct=assert_function)
def test_from_room_to_hallway(diagram: str, i_movable: int, hallway_spot: int) -> int:
    return StateII.parse(diagram).move_from_room_to_hallway(i_movable, hallway_spot)


Test Part I Test Room to hallway to the left passed, for test_from_room_to_hallway.
Test Part I Test Room to hallway to the left passed, for test_from_room_to_hallway.
Test Part II test 1 passed, for test_from_room_to_hallway.
Test Part II test 2 passed, for test_from_room_to_hallway.
Success


In [134]:
tests_from_hallway_to_room = [
    {
        "name": "Part I Test 1",
        "diagram": f"""
                 #############
                 #B........DD#
                 ###C#.#B#.###
                   #A#.#C#A#      
                   #########    
                 """,
        "i_movable": 3,
        "expected": StateII(
            energy=60,
            movable=(
                (C, 0, 0),
                (B, 2, 0),
                (A, 3, 1),
                (D, -1, 9),
                (D, -1, 10),
            ),
            hallway=tuple({9: D, 10: D}.get(i, E) for i in range(11)),
            rooms=(
                (C,),
                (E,),
                (B,),
                (E, A),
            ),
            n=2,
        ),
    },
    {
        "name": "Part I test 2",
        "diagram": f"""
                  #############
                  #B.........D#
                  ###B#.#C#D###
                    #A#.#C#A#
                    #########    
                 """,
        "i_movable": 2,
        "expected": StateII(
            energy=60,
            movable=(
                (B, 0, 0),
                (D, 3, 0),
                (D, -1, 10),
            ),
            hallway=tuple({10: D}.get(i, E) for i in range(11)),
            rooms=(
                (B,),
                (E,),
                tuple(),
                (D, A),
            ),
            n=2,
        ),
    },
    {
        "name": "Part II test 1",
        "diagram": f"""
                #############
                #AA.D.B.B.BD#
                ###B#.#.#.###
                  #D#.#C#.#
                  #D#.#C#C#
                  #A#.#C#A#
                  #########
                 """,
        "i_movable": 5,
        "expected": StateII(
            energy=50,
            movable=(
                (B, 0, 0),
                (C, 3, 2),
                (A, -1, 0),
                (A, -1, 1),
                (D, -1, 3),
                (B, -1, 7),
                (B, -1, 9),
                (D, -1, 10),
            ),
            hallway=tuple(
                {0: A, 1: A, 3: D, 7: B, 9: B, 10: D}.get(i, E) for i in range(11)
            ),
            rooms=(
                (B, D, D),
                (E, E, E),
                (E,),
                (E, E, C, A),
            ),
            n=4,
        ),
    },
]


@test(tests=tests_from_hallway_to_room)
def test_from_hallway_to_room(diagram: str, i_movable: int) -> int:
    return StateII.parse(diagram).from_hallway_to_room(i_movable)


Test Part I Test 1 passed, for test_from_hallway_to_room.
Test Part I test 2 passed, for test_from_hallway_to_room.
Test Part II test 1 passed, for test_from_hallway_to_room.
Success


In [135]:
tests_from_room_to_room = [
    {
        "name": "Part I Test 1",
        "diagram": f"""
                 #############
                 #B.........D#
                 ###.#C#B#.###
                   #A#D#C#A#      
                   #########    
                 """,
        "i_movable": 2,
        "expected": StateII(
            energy=9,
            movable=(
                (C, 1, 0),
                (B, 2, 0),
                (B, -1, 0),
                (D, -1, 10),
            ),
            hallway=(B,) + tuple(E for _ in range(1, 10)) + (D,),
            rooms=(
                tuple(),
                (C, D),
                (B,),
                (E, E),
            ),
            n=2,
        ),
    },
    {
        "name": "Part I test 2",
        "diagram": f"""
                  #############
                  #B.........D#
                  ###B#.#C#D###
                    #A#.#C#A#
                    #########    
                 """,
        "i_movable": 0,
        "expected": StateII(
            energy=50,
            movable=(
                (D, 3, 0),
                (B, -1, 0),
                (D, -1, 10),
            ),
            hallway=tuple({0: B, 10: D}.get(i, E) for i in range(11)),
            rooms=(
                (E,),
                (E,),
                tuple(),
                (D, A),
            ),
            n=2,
        ),
    },
    {
        "name": "Part I Test 3",
        "diagram": f"""
                 #############
                 #..........D#
                 ###.#C#B#A###
                   #A#D#C#B#      
                   #########    
                 """,
        "i_movable": 2,
        "expected": StateII(
            energy=8,
            movable=(
                (C, 1, 0),
                (B, 2, 0),
                (D, -1, 10),
                (B, 3, 1),
            ),
            hallway=tuple(E for _ in range(10)) + (D,),
            rooms=(
                tuple(),
                (C, D),
                (B,),
                (E, B),
            ),
            n=2,
        ),
    },
    {
        "name": "Part II test 1",
        "diagram": f"""
                  #############
                  #AA.....B.BD#
                  ###B#.#.#.###
                    #D#C#.#.#
                    #D#B#C#C#
                    #A#D#C#A#
                    #########    
                 """,
        "i_movable": 1,
        "expected": StateII(
            energy=600,
            movable=(
                (B, 0, 0),
                (C, 3, 2),
                (A, -1, 0),
                (A, -1, 1),
                (B, -1, 7),
                (B, -1, 9),
                (D, -1, 10),
                (B, 1, 2),
            ),
            hallway=tuple({0: A, 1: A, 7: B, 9: B, 10: D}.get(i, E) for i in range(11)),
            rooms=(
                (B, D, D),
                (E, E, B, D),
                (E,),
                (E, E, C, A),
            ),
            n=4,
        ),
    },
]


@test(tests=tests_from_room_to_room)
def test_from_room_to_room(diagram: str, i_movable: int) -> int:
    return StateII.parse(diagram).from_room_to_room(i_movable)


Test Part I Test 1 passed, for test_from_room_to_room.
Test Part I test 2 passed, for test_from_room_to_room.
Test Part I Test 3 passed, for test_from_room_to_room.
Test Part II test 1 passed, for test_from_room_to_room.
Success


In [136]:
tests_ia_goal_part_II = [
    {
        "name": "example part I",
        "diagram": f"""
                 #############
                 #...........#
                 ###B#C#B#D###
                   #A#D#C#A#      
                   #########    
                 """,
        "expected": False,
    },
    {
        "name": "goal state part I",
        "diagram": f"""
                   #############
                   #...........#
                   ###A#B#C#D###
                     #A#B#C#D#
                     #########""",
        "expected": True,
    },
    {
        "name": "example part II",
        "diagram": f"""
                 #############
                 #...........#
                 ###B#C#B#D###
                   #D#C#B#A#
                   #D#B#A#C#
                   #A#D#C#A#      
                   #########    
                 """,
        "expected": False,
    },
    {
        "name": "Goal state part II",
        "diagram": f"""
                 #############
                 #...........#
                 ###A#B#C#D###
                   #A#B#C#D#
                   #A#B#C#D#
                   #A#B#C#D#      
                   #########    
                 """,
        "expected": True,
    },
]


@test(tests=tests_ia_goal_part_II)
def test_ia_goal_part_II(diagram: str) -> bool:
    return StateII.parse(diagram).is_goal()


Test example part I passed, for test_ia_goal_part_II.
Test goal state part I passed, for test_ia_goal_part_II.
Test example part II passed, for test_ia_goal_part_II.
Test Goal state part II passed, for test_ia_goal_part_II.
Success


In [137]:
tests_parse = [
    {
        "name": "example part I",
        "diagram": f"""
                 #############
                 #...........#
                 ###B#C#B#D###
                   #A#D#C#A#      
                   #########    
                 """,
        "expected": StateII(
            energy=0,
            movable=(
                (B, 0, 0),
                (C, 1, 0),
                (B, 2, 0),
                (D, 3, 0),
            ),
            hallway=tuple(E for _ in range(11)),
            rooms=(
                (B,),
                (C, D),
                (B,),
                (D, A),
            ),
            n=2,
        ),
    },
    {
        "name": "puzzle part I",
        "diagram": f"""
                   #############
                   #...........#
                   ###D#D#C#C###
                     #B#A#B#A#
                     #########""",
        "expected": StateII(
            energy=0,
            movable=(
                (D, 0, 0),
                (D, 1, 0),
                (C, 2, 0),
                (C, 3, 0),
            ),
            hallway=tuple(E for _ in range(11)),
            rooms=(
                (D, B),
                (D, A),
                (C, B),
                (C, A),
            ),
            n=2,
        ),
    },
    {
        "name": "Goal state part I",
        "diagram": f"""
                   #############
                   #...........#
                   ###A#B#C#D###
                     #A#B#C#D#
                     #########""",
        "expected": StateII(
            energy=0,
            movable=tuple(),
            hallway=tuple(E for _ in range(11)),
            rooms=(
                tuple(),
                tuple(),
                tuple(),
                tuple(),
            ),
            n=2,
        ),
    },
    {
        "name": "example part II",
        "diagram": f"""
                 #############
                 #...........#
                 ###B#C#B#D###
                   #D#C#B#A#
                   #D#B#A#C#
                   #A#D#C#A#      
                   #########    
                 """,
        "expected": StateII(
            energy=0,
            movable=(
                (B, 0, 0),
                (C, 1, 0),
                (B, 2, 0),
                (D, 3, 0),
            ),
            hallway=tuple(E for _ in range(11)),
            rooms=(
                (B, D, D),
                (C, C, B, D),
                (B, B, A),
                (D, A, C, A),
            ),
            n=4,
        ),
    },
    {
        "name": "example part II",
        "diagram": f"""
                 #############
                 #...........#
                 ###A#B#C#D###
                   #A#B#C#D#
                   #A#B#C#D#
                   #A#B#C#D#      
                   #########    
                 """,
        "expected": StateII(
            energy=0,
            movable=tuple(),
            hallway=tuple(E for _ in range(11)),
            rooms=(
                tuple(),
                tuple(),
                tuple(),
                tuple(),
            ),
            n=4,
        ),
    },
    {
        "name": "Hallway not empty part I",
        "diagram": f"""
                 #############
                 #B.........D#
                 ###.#C#B#.###
                   #A#D#C#A#      
                   #########    
                 """,
        "expected": StateII(
            energy=0,
            movable=(
                (C, 1, 0),
                (B, 2, 0),
                (A, 3, 1),
                (B, -1, 0),
                (D, -1, 10),
            ),
            hallway=(B,) + tuple(E for _ in range(1, 10)) + (D,),
            rooms=(
                (E,),
                (C, D),
                (B,),
                (E, A),
            ),
            n=2,
        ),
    },
]


@test(tests=tests_parse)
def test_parse(diagram: str) -> int:
    return StateII.parse(diagram)


Test example part I passed, for test_parse.
Test puzzle part I passed, for test_parse.
Test Goal state part I passed, for test_parse.
Test example part II passed, for test_parse.
Test example part II passed, for test_parse.
Test Hallway not empty part I passed, for test_parse.
Success


In [138]:
# puzzle part II
puzzle = f"""
    #############
    #...........#
    ###D#D#C#C###
      #D#C#B#A#
      #D#B#A#C#
      #B#A#B#A#
      #########"""

print(f"Part II: { StateII.parse(puzzle).solve()}")

Part II: 43117


<link href="style.css" rel="stylesheet"></link>
<main>
<article><p>That's the right answer!  You are <span class="day-success">one gold star</span> closer to finding the sleigh keys.</p><p>You have completed Day 23! You can <span class="share">[Share<span class="share-content">on
  <a href="https://bsky.app/intent/compose?text=I+just+completed+%22Amphipod%22+%2D+Day+23+%2D+Advent+of+Code+2021+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2021%2Fday%2F23" target="_blank">Bluesky</a>
  <a href="https://twitter.com/intent/tweet?text=I+just+completed+%22Amphipod%22+%2D+Day+23+%2D+Advent+of+Code+2021&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2021%2Fday%2F23&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I+just+completed+%22Amphipod%22+%2D+Day+23+%2D+Advent+of+Code+2021+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2021%2Fday%2F23';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this victory or <a href="/2021">[Return to Your Advent Calendar]</a>.</p></article>
</main>


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>43117</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
